In [28]:
from itertools import product

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import plotly.express as px
from IPython.display import display

# Setup

## Constants

In [2]:
CAT_LEN = 10
LABELS = list('etainoshrd')

char_to_label = { char: idx for idx, char in enumerate(LABELS) }
label_to_char = { idx: char for char, idx in char_to_label.items() }

In [3]:
def read_array(file):
    return np.loadtxt(file)

def get_features(num, split='train'):
    '''
    Return train/test data as ndarray
    Shape: (seq_len, feature_dim)
    '''
    return read_array(f'./data/{split}_img{num}.txt')

def normalize(ndarray, axis=0):
    return ndarray / ndarray.sum(axis=axis)

In [4]:
feature_params = read_array('./model/feature-params.txt')
print('Feature params shape', feature_params.shape)

transition_params = read_array('./model/transition-params.txt')
print('Transition params shape', transition_params.shape)

Feature params shape (10, 321)
Transition params shape (10, 10)


In [5]:
def get_word(labels):
    return ''.join([label_to_char[label] for label in labels])

def get_labels(split='train'):
    labels = []
    with open(f'./data/{split}_words.txt') as file:
        for word in file:
            labels.append([char_to_label[char] for char in word.rstrip('\n')])
    return labels

train_labels = get_labels(split='train')
test_labels = get_labels(split='test')

print('theodore', train_labels[0])

theodore [1, 7, 0, 5, 9, 5, 8, 0]


# 1. Exhaustive Inference

The CRF also contains one transition parameter $W_{cc′}$ for each pair of character labels c and c .
The transition parameters encode the "compatibility" between adjacent character labels in the word

In [6]:
def calc_feat_potential(features, params):
    '''
        features: ndarray of shape (length, dims). dims = 231
        params: ndarray of shape (categories, dims) categories = 10
        
        return: potentials of shape (seq length, categories)
    '''
    return np.exp(features @ params.T)

In [7]:
def calc_tran_potential(params):
    '''
        Calculates the transition potentials.
        params: ndarray of shape (categories, categories)
        
        Return: ndarray of shape (cat, cat) 
        potential[y_1,y_2] will give the potential from label y_1 to y_2
    '''
    return np.exp(params)

In [8]:
def get_potens(test_case=1):
    features = get_features(test_case, split='test')
    
    feature_potens = calc_feat_potential(features, feature_params)
    trans_potens = calc_tran_potential(transition_params)
    
    return feature_potens, trans_potens

## 1.1 Feature potentials of $test_0$

In [9]:
test_1 = get_features(1, split='test')

# Calculate potentials in log space
log_potentials = np.log(calc_feat_potential(test_1, feature_params))
log_potentials

array([[ -7.644354,  18.46838 ,  -6.328565,  10.422494,  -4.967162,
         -1.934011,  -0.945172,  -5.657108,   5.395295,  -6.809825],
       [ -4.074483,   5.744835,   1.176369,  -1.793123,  -1.212227,
         -1.78488 ,  -8.299875,   3.095185,   6.806588,   0.341613],
       [-10.208138,   0.897342,  17.191008, -12.017674,   5.57936 ,
         -0.594043, -21.426366,   9.148904,   9.482416,   1.947176],
       [  6.464858,  24.531253, -13.342905,   5.871221, -10.95484 ,
        -11.496499,  -5.494649,  -7.195623,   8.045658,   3.571496]])

## 1.2 Energy Calculation

Energy is a function which measures the “goodness” (or badness) of each possible configuration of the random variables (the lower the engergy, the better the configuration).
Define energy as the negative logarithm of (possibly unnormalized) probability.

For the first three test words, compute the value of the negative energy of the true label sequence after conditioning on the corresponding observed image sequence.

In [10]:
def calc_energy(feat_potens, trans_params, true_labels):
    '''
        Calculate the **negative** energy of a distribution. Energy is defined as negative logarithm of probability
        
        feat_poten: ndarray of shape (seq length, categories): Feature potentials
        trans_params: ndarray of shape (categories, categories): Transition parameters
        
        returns: ndarray of shape (categories)
    '''
    # Calculate the feature potentials of true labels
    feat_comp = np.log(feat_potens[range(len(true_labels)), true_labels]).sum()
    
    # Calculate the transition potentials of true labels
    trans_comp = trans_params[true_labels[:-1], true_labels[1:]].sum()
    
    energy = feat_comp + trans_comp
    
    return energy

In [11]:
# Energy for first 3 test words
for i in range(3):
    features = get_features(i + 1, split='test')
    feature_potentials = calc_feat_potential(features, feature_params)
    energy = calc_energy(feature_potentials, transition_params, test_labels[i])
    print(f'Test word {i + 1}:', energy)

Test word 1: 63.979336
Test word 2: 89.61093
Test word 3: 96.940634


## 1.3, 1.4:  Log Partition function, Most-likely

Calculate $\log Z$

In [12]:
def calc_potential_for_seq(sequence, feat_potens, trans_potens):
    potential = 1
    # Feature poten
    for seq_i, label in enumerate(sequence):
        potential *= feat_potens[seq_i, label]

    # Transition poten
    for seq_i, label in enumerate(sequence[:-1]):
        next_label = sequence[seq_i + 1]
        potential *= trans_potens[label, next_label]

    return potential

In [13]:
def calc_log_partition(features):
    '''
     Get a (categories,) array and sum it up. Since x is observed
    '''
    
    feat_potens = calc_feat_potential(features, feature_params)
    trans_potens = calc_tran_potential(transition_params)
    
    cat_len = 10
    seq_len, dim_len = features.shape
    
    possiblities = [range(cat_len)] * seq_len

    log_partition = 0
    most_likely = (None, float('-inf'))
    
    # Go over all possible sequences
    for sequence in product(*possiblities):
        potential = calc_potential_for_seq(sequence, feat_potens, trans_potens)

        if potential > most_likely[1]:
            most_likely = (sequence, potential)

        log_partition += potential
        
    print('Most Likely: ', get_word(most_likely[0]), 'Prob: ', most_likely[1] / log_partition)
    
    return np.log(log_partition)

In [14]:
# Log partition for first 3 test words
for i in range(3):
    features = get_features(i+1, split='test')
    log_partition = calc_log_partition(features)
    print(f'Test {i+1}:', log_partition)

Most Likely:  trat Prob:  0.7958187630401371
Test 1: 67.60187580368476
Most Likely:  hire Prob:  0.9965204924093368
Test 2: 89.61441557515604
Most Likely:  riser Prob:  0.9370071414912935
Test 3: 103.52757237511717


## 1.5 Marginal Probabilities

In [15]:
def get_marginal():
    features = get_features(1, split='test')
    feat_potens = calc_feat_potential(features, feature_params)
    trans_potens = calc_tran_potential(transition_params)

    cat_len = 10
    seq_len, dim_len = features.shape
    
    possiblities = [range(cat_len)] * seq_len
    probs = np.zeros((seq_len, cat_len))
    
    for sequence in product(*possiblities):
        potential = calc_potential_for_seq(sequence, feat_potens, trans_potens)
        
        for seq_i, label in enumerate(sequence):
            probs[seq_i, label] += potential

    probs = probs / probs.sum(axis=1).reshape(-1, 1)

    return probs

probs = get_marginal()
probs.T

array([[7.22268182e-12, 1.26583711e-05, 1.13213821e-12, 8.86828033e-09],
       [9.99524645e-01, 1.72473165e-01, 2.29451201e-08, 9.99999919e-01],
       [2.62616683e-11, 2.73136691e-03, 9.99458877e-01, 2.13566003e-17],
       [4.72721273e-04, 1.75283394e-04, 1.61185516e-13, 7.40542921e-09],
       [7.15554575e-11, 2.00735648e-04, 3.69756698e-06, 3.29004119e-16],
       [2.11384828e-09, 1.40047474e-04, 1.76109354e-08, 1.44100307e-16],
       [3.29598967e-09, 1.06460709e-07, 5.17214285e-18, 5.37109230e-14],
       [4.34926966e-11, 2.67352876e-02, 2.83525328e-04, 1.31780714e-14],
       [2.62808981e-06, 7.96595064e-01, 2.53764930e-04, 6.39397930e-08],
       [1.06936989e-11, 9.36285224e-04, 9.46377346e-08, 6.37362801e-10]])

# 2. Sum-Message Passing

In this question, you will implement the sum-product inference algorithm for the CRF model. The code packages provides a pre-trained model for the OCR task including the feature parameters (feature-params.txt) and the label-label transition parameters (transition-params.txt). Use these parameters to answer the following questions.

## 2.1 Log-space messages

For the first test word only, condition on the observed image sequence to obtain a chain-structured Markov network. Compute the log-space messages 

$m_{1 \rightarrow 2}(Y_2)$, 

$m_{3 \rightarrow 1}(Y_1)$, 

$m_{2 \rightarrow 3}(Y_3)$, 

$m_{3 \rightarrow 2}(Y_2)$

Report the value of each message in a table. 10 values per message

In [16]:
def calc_messages(feature_potens, trans_potens, forward=True):
    '''
        Calculate the **log-space** messages that are passed from i -> j
        
        feature_potens: ndarray (seq_len, cat_len)
        trans_potens: ndarray (cat_len, cat_len)
        
        forward: Compute the messages forward 1 -> 2, or backward 2 -> 1
        
        Returns: msg: Dict[(from, to)] -> ndarray (categories,)
    '''
    msg = dict()

    seq_len, cat_len = feature_potens.shape

    if forward:
        msg[0, 1] = np.ones(cat_len)

        for seq_i in range(1, seq_len):
            msg[seq_i, seq_i + 1] = np.zeros(cat_len)

            for cat_i in range(cat_len):
                for cat_j in range(cat_len):
                    msg[seq_i, seq_i + 1][cat_i] += (feature_potens[seq_i - 1, cat_j] 
                                       * trans_potens[cat_i, cat_j] 
                                       * msg[seq_i - 1, seq_i][cat_j])
    else:
        msg[seq_len + 1, seq_len] = np.ones(cat_len)

        for seq_i in range(seq_len, 1, -1):
            msg[seq_i, seq_i - 1] = np.zeros(cat_len)

            for cat_i in range(cat_len):
                for cat_j in range(cat_len):
                    msg[seq_i, seq_i - 1][cat_i] += (feature_potens[seq_i - 1, cat_j] 
                                              * trans_potens[cat_i, cat_j]
                                              * msg[seq_i + 1, seq_i][cat_j])

    return msg

In [17]:
def get_messages(feat_potens, trans_potens):
    return (calc_messages(feat_potens, trans_potens, forward=True), 
            calc_messages(feat_potens, trans_potens, forward=False))

In [18]:
potens = get_potens(test_case=1)
msg_f, msg_b = get_messages(*potens)

df = pd.DataFrame(np.log([msg_f[1, 2], msg_b[2, 1], msg_f[2, 3], msg_b[3, 2]]), columns=LABELS)
df.index = ['1 - 2', '2 - 1', '2 - 3', '3 - 2']

df.T

,1 - 2,2 - 1,2 - 3,3 - 2
e,18.589345,49.592435,25.651079,41.809822
t,17.815295,49.133020,25.236859,42.284232
a,18.749373,49.567530,25.598383,41.773180
i,18.522734,49.522377,25.577943,42.223158
n,18.180753,49.208489,25.271637,42.119828
o,18.677310,49.561131,25.601245,41.835916
s,18.091288,49.016488,25.071460,41.754973
h,18.834070,49.400556,25.388027,42.050850
r,18.363419,49.357328,25.414512,42.204460
d,18.216396,49.150334,25.202644,42.070277


## 2.2 Marginal Probabilties

For the first test word only, use the computed messages to compute marginal probability distributions. Report single variable marginals over each position in the word as a table. Represent pairwise marginals over each adjacent node pairs as three tables, and report only the 3 × 3 block of entries between the labels “t,a,h” in each table.

In [19]:
def calc_marginal_single(feature_potens, trans_potens, msg_f, msg_b):    
    seq_len, cat_len = feature_potens.shape
    
    marginals = []
    for seq_i in range(1, seq_len + 1):
        p_i = normalize(feature_potens[seq_i - 1,:] * msg_f[seq_i - 1,seq_i] * msg_b[seq_i + 1, seq_i])
        marginals.append(np.expand_dims(p_i, axis=0))
    
    marginals = np.concatenate(marginals)

    return marginals

potens = get_potens(test_case=1)

messages = get_messages(*potens)
marginals = calc_marginal_single(*potens, *messages)

pd.DataFrame(marginals, columns=LABELS).T

,0,1,2,3
e,7.222682e-12,1.265837e-05,1.132138e-12,8.868280e-09
t,9.995246e-01,1.724732e-01,2.294512e-08,9.999999e-01
a,2.626167e-11,2.731367e-03,9.994589e-01,2.135660e-17
i,4.727213e-04,1.752834e-04,1.611855e-13,7.405429e-09
n,7.155546e-11,2.007356e-04,3.697567e-06,3.290041e-16
o,2.113848e-09,1.400475e-04,1.761094e-08,1.441003e-16
s,3.295990e-09,1.064607e-07,5.172143e-18,5.371092e-14
h,4.349270e-11,2.673529e-02,2.835253e-04,1.317807e-14
r,2.628090e-06,7.965951e-01,2.537649e-04,6.393979e-08
d,1.069370e-11,9.362852e-04,9.463773e-08,6.373628e-10


### Marginal Pair probabilities

In [20]:
def calc_marginals_pair(feature_potens, trans_potens, msg_f, msg_b):
    seq_len, cat_len = feature_potens.shape

    marginals = []
    
    # There are seq_len - 1 pairs
    for pair_i in range(seq_len - 1):
        right_scores = (feature_potens[pair_i + 1,:] * msg_b[pair_i + 3,pair_i + 2]).reshape((1, -1))
        left_scores = (feature_potens[pair_i,:] * msg_f[pair_i, pair_i + 1]).reshape((-1,1))

        scores = left_scores * trans_potens * right_scores
        scores = scores / scores.sum()

        marginals.append(np.expand_dims(scores, axis=0))

    marginals = np.concatenate(marginals)
        
    return marginals
    
potens = get_potens(test_case=1) 
messages = get_messages(*potens)

marginals = calc_marginals_pair(*potens, *messages)

for marginal in marginals:
    df = pd.DataFrame(marginal, columns=LABELS)
    df.index = LABELS
    display(df.loc[['t','h','a'], ['t', 'h', 'a']])

,t,h,a
t,1.723604e-01,2.672975e-02,2.730539e-03
h,1.590427e-11,5.389681e-13,7.200092e-14
a,7.465838e-12,3.308640e-13,2.785953e-14


,t,h,a
t,2.231417e-09,0.000066,0.172371
h,1.210440e-09,0.000008,0.026720
a,1.499698e-10,0.000001,0.002729


,t,h,a
t,2.294512e-08,1.058097e-21,2.079552e-24
h,2.835253e-04,2.857058e-18,7.343193e-21
a,9.994588e-01,1.317085e-14,2.133678e-17


## 2.3 Inference over test

In [21]:
# TODO: Get predictions for first 3 test words
def predict_crf(test_case=1):
    feat_potens, trans_potens = get_potens(test_case)
    msg_f, msg_b = get_messages(feat_potens, trans_potens)

    seq_len, cat_len = feat_potens.shape
    
    # Shape (seq_len, 10)
    single_marginals = calc_marginal_single(feat_potens, trans_potens, msg_f, msg_b)

    # Shape (seq_len, 10, 10)
    pair_marginals = calc_marginals_pair(feat_potens, trans_potens, msg_f, msg_b)
    
    # Pad pair_marginals with ones to simplify calculation
    pair_marginals = np.vstack((np.ones((1, 10, 10)), pair_marginals))
        
    prediction = []
    cat_label = 0
    for seq_i in range(seq_len):
        cat_label = np.argmax(pair_marginals[seq_i, cat_label] * single_marginals[seq_i])
        prediction.append(label_to_char[cat_label])
    
    return ''.join(prediction)

In [22]:
# Predictions over first 5 test words
with open('./data/test_words.txt') as f:
    num = 10
    for idx, actual in enumerate(f.readlines()):
        if idx == num:
            break
        print('Actual: ', actual.rstrip(), 'Pred: ', predict_crf(test_case=idx+1))

Actual:  that Pred:  trat
Actual:  hire Pred:  hire
Actual:  rises Pred:  riser
Actual:  edison Pred:  edison
Actual:  shore Pred:  shore
Actual:  tenth Pred:  tenth
Actual:  not Pred:  hot
Actual:  tests Pred:  tests
Actual:  trains Pred:  trains
Actual:  order Pred:  order


In [23]:
words = None
with open('./data/test_words.txt') as f:
    words = [line.rsplit('\n')[0] for line in f.readlines()]

correct = 0
total = 0

for i in range(1, 201):
    prediction = predict_crf(test_case=i)
    for char_p, char_t in zip(list(prediction), list(words[i - 1])):
        if char_p == char_t: 
            correct += 1
        total += 1

print('Accuracy: ', correct / total)

Accuracy:  0.8991674375578168


# 3. Maximum Likelihood Learning Derivation

In this problem, you will derive the maximum likelihood learning algorithm for conditional random field model

## 3.5 Average log likelihood

Using a dataset consisting of the first 50 training data cases only, compute the average log likelihood of the true label sequences given the image sequences using the supplied model parameters.

In [24]:
def calc_log_partition_v2(feat_potens, msg):
    log_partition = feat_potens[0] * msg[2,1]
    log_partition = log_partition.sum()
    log_partition = np.log(log_partition)
    
    return log_partition

# Read training data
def compute_likelihood(num):
    # Get features
    features = get_features(num + 1, split='train') 
    
    feat_potens = calc_feat_potential(features, feature_params)
    trans_potens = calc_tran_potential(transition_params)
    
    msg_f, msg_b = get_messages(feat_potens, trans_potens)

    seq_len, cat_len = feat_potens.shape
    
    log_partition = calc_log_partition_v2(feat_potens, msg_b)
    
    labels = train_labels[num]    
    feat_potens_val = np.log(feat_potens[range(seq_len),  labels]).sum()
    trans_potens_val = transition_params[labels[:-1], labels[1:]].sum()
    
    likelihood = feat_potens_val + trans_potens_val - log_partition
    
    return likelihood

In [25]:
# Averate likelihood
num = 50
avg_likelihood = np.mean([compute_likelihood(i) for i in range(num)])
avg_likelihood

-4.583959036355722

# 4. Numerical Optimizaion warm-up

In the next assignment, you will im- plement the above learning algorithm using a numerical optimizer to maximize the log likelihood. In this question, you will experiment with optimizing a basic function.

In [26]:
from scipy.optimize import fmin_l_bfgs_b
from scipy.optimize import minimize

In [27]:
def objective(vec):
    x, y = vec
    return (1 - x) ** 2 + 100*((y - x ** 2) ** 2)

def calc_jacardian(vec):
    '''Gradient of the objective function'''
    x, y = vec

    return -np.array([
        2 * (1 - x) + 400 * x * (y - x ** 2),
        -200*(y - x ** 2)
    ])

xy = np.array([1, -1])

# Calculate gradient numerically
# minimize(objective, xy)

minimize(objective, xy, jac=calc_jacardian, method='L-BFGS-B')

      fun: 1.2586977413785688e-16
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 2.71276770e-07, -1.44232692e-07])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 32
      nit: 26
   status: 0
  success: True
        x: array([0.99999999, 0.99999998])

# Assignment 3

Use average log conditional likelihood as the objective function. Implement the objective function, and also its gradient functions (the computation of the partial derivatives of the objective function with respect to each parameter), as derived in Assignment 2. Use your implementation of the sum-product message passing algorithm from Assignment 2 as a subroutine to make your objective and gradient function implementations computationally tractable. Implement the learning algorithm for CRFs by using the numerical optimizer you selected in Assignment 2 to maximize the log conditional likelihood function. Use the first 50, 100, 150, 200, 250, 300, 350 and 400 training data cases to train eight separate CRF models. Answer the following questions.

In [ ]:
batch_sizes = [50, 100, 150, 200, 250, 300, 350, 400]

for batch_size in batch_sizes:
    # Read batch_size training examples
    

## 1.1

Record the total training time in seconds for each of the above training data set sizes. Report your results as a line graph of time in seconds versus training set size. Make sure to label the axes of your plot.

## 1.2

Evaluate the prediction error of each model on the complete test set. As in Assignment 2, predict the character with the highest marginal probability for each position of each test word. Report the error rate averaged over all predicted characters in all test words for each model. Summarize your test error results in a line graph showing prediction error versus training set size. Make sure to label the axes of your plot.

## 1.3

Evaluate the average conditional log likelihood of the complete test set under each model (for each model, this will be an average of the per-word conditional log likelihoods for each word in the test set). Summarize your results in a line graph showing average conditional log likelihood versus training set size. Make sure to label the axes of your plot.